In [11]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S4'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 1912
Total Tokens Found = 24340
Total Tokens Output = 24287
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 1912
Total Tokens= 24287
Unique Tokens= 5182
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  3
Maximum Frequency of Error Token =  3
References Processed= 1912
Sorted Token Size = 5182
Clean Token Size = 2394
*Stop Replacements here
References Processed =  1912
Total Replacement Pairs = 113
Tokens Read = 24287
Tokens Changed =  164
References Changed = 156


In [12]:
status="done"

In [13]:
print(status)

done


In [14]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [15]:
import numpy as np

Create a function of the parameters that returns the F score

In [16]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [17]:
entropy_F1(mu=0.5,muIncr=0.05,beta=33,sigma=449,epsilon=31.8,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2711
NewMatrix Iterateblocks Total Time = 2.42470367718488


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2077
NewMatrix Iterateblocks Total Time = 1.8216272424906492


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1176
NewMatrix Iterateblocks Total Time = 1.1315043196082115


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 454
NewMatrix Iterateblocks Total Time = 0.7782448390498757


>>>>>>>>>>>>
Starting Iteration mu= 0.7000000000000002
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!


0.49909209

Create functions for variable transformation.

In [18]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [19]:
inv_softplus(1)

0.541324854612918

In [20]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [21]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [22]:
import os, sys

In [23]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [24]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [25]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [26]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


-0.0036316439914252374

In [35]:
optimizer=None

In [36]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(500)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(500)),
           'mu_logit': (logit(0.5), logit(0.8)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(500)),
           'epsilonIterate_logit': (logit(0), logit(0.25))}

In [37]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
)

In [38]:
optimizer.probe(
    params={"beta_sp": inv_softplus(12), 
            "sigma_sp": inv_softplus(22),
            "mu_logit":logit(0.5),
            'epsilonIterate_logit': logit(0),
            "muIncr_logit":logit(0.1),
            "epsilon_sp":inv_softplus(3.1)},
    lazy=True,
)

In [39]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
)

|   iter    |  target   |  beta_sp  | epsilo... | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------------------
Mu:0.50|Mu+:0.10|Beta:12.00|Sigma:22.00|Epsilon:3.10|EpIter:0.00
|  1        |  2.34     |  12.0     | -124.3    |  3.054    | -2.197    |  0.0      |  22.0     |
Mu:0.55|Mu+:0.03|Beta:208.00|Sigma:46.00|Epsilon:1.04|EpIter:0.00
|  2        |  1.007    |  208.8    | -35.57    |  0.5985   | -3.625    |  0.2034   |  46.66    |
Mu:0.64|Mu+:0.05|Beta:93.00|Sigma:342.00|Epsilon:198.71|EpIter:0.00
|  3        |  0.7104   |  93.57    | -81.75    |  198.7    | -2.866    |  0.5811   |  342.8    |
Mu:0.64|Mu+:0.08|Beta:102.00|Sigma:279.00|Epsilon:14.22|EpIter:0.00
|  4        |  1.126    |  102.7    | -16.12    |  14.22    | -2.444    |  0.5785   |  279.6    |
Mu:0.61|Mu+:0.18|Beta:70.00|Sigma:346.00|Epsilon:400.48|EpIter:0.00
|  5        | -0.2025   |  70.66    | -99.93    |  400.5    | -1.488    |  0.

|  36       |  2.096    |  111.1    | -80.81    |  109.1    | -3.664    |  1.229    |  117.6    |
Mu:0.80|Mu+:0.20|Beta:107.00|Sigma:128.00|Epsilon:110.69|EpIter:0.00
|  37       |  1.953    |  107.8    | -76.12    |  110.7    | -1.386    |  1.386    |  128.1    |
Mu:0.58|Mu+:0.14|Beta:163.00|Sigma:137.00|Epsilon:81.31|EpIter:0.00
|  38       | -4.278    |  163.3    | -33.06    |  81.31    | -1.835    |  0.3048   |  137.2    |
Mu:0.69|Mu+:0.01|Beta:162.00|Sigma:127.00|Epsilon:111.09|EpIter:0.00
|  39       |  1.423    |  162.4    | -26.52    |  111.1    | -4.452    |  0.7999   |  128.0    |
Mu:0.80|Mu+:0.01|Beta:215.00|Sigma:185.00|Epsilon:136.57|EpIter:0.00
|  40       |  1.923    |  215.5    | -64.65    |  136.6    | -4.595    |  1.386    |  185.3    |
Mu:0.73|Mu+:0.03|Beta:115.00|Sigma:182.00|Epsilon:123.51|EpIter:0.00
|  41       |  1.847    |  115.4    | -52.61    |  123.5    | -3.621    |  0.9726   |  182.3    |
Mu:0.80|Mu+:0.20|Beta:107.00|Sigma:120.00|Epsilon:104.28|EpIter:0.00

In [40]:
optimizer.max

{'target': 2.3403714494284453,
 'params': {'beta_sp': 11.999993855768771,
  'epsilonIterate_logit': -124.33959502167846,
  'epsilon_sp': 3.053904539340399,
  'muIncr_logit': -2.197224577336219,
  'mu_logit': 0.0,
  'sigma_sp': 21.99999999972105}}

In [41]:
inv_logit(optimizer.max["target"])

0.91216585

In [42]:
params=optimizer.max["params"]

In [43]:
softplus(params["beta_sp"])

12.0

In [44]:
softplus(params["sigma_sp"])

22.0

In [45]:
inv_logit(params["mu_logit"])

0.5

In [46]:
inv_logit(params["muIncr_logit"])

0.10000000000000003

In [47]:
inv_logit(params["epsilonIterate_logit"])

0

In [48]:
softplus(params["epsilon_sp"])

3.1

In [50]:
entropy_F1(mu=0.5,
           muIncr=0.1,
           beta=12,
           sigma=22,
           epsilon=3.1,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2529
NewMatrix Iterateblocks Total Time = 1.3589080031961203


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2030
NewMatrix Iterateblocks Total Time = 1.0290703903883696


>>>>>>>>>>>>
Starting Iteration mu= 0.7
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1255
NewMatrix Iterateblocks Total Time = 0.9258546400815248


>>>>>>>>>>>>
Starting Iteration mu= 0.7999999999999999
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 538
NewMatrix Iterateblocks Total Time = 0.5582271860912442


>>>>>>>>>>>>
Starting Iteration mu= 0.8999999999999999
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference B

0.91630435